# Monoidal Trees

Based on Brent A. Yorgey. 2012. Monoids: theme and variations (functional pearl). SIGPLAN Not. 47, 12 (December 2012), 105–116. https://doi.org/10.1145/2430532.2364520

Also see https://github.com/diagrams/dual-tree


In [12]:
import $ivy.`org.typelevel::cats-core:2.1.0`

import $ivy.$                               

In [13]:
import cats._
import cats.implicits._

trait Action[M, A] extends Monoid[M] {
  def act(a: A): A
}

sealed trait MTree[D, U, L] {
  def u: U
} 

case class Leaf[U, L](u: U, l: L) extends MTree[Nothing, U, L]

case class Concat[D, U, L](u: U, ts: List[MTree[D, U, L]]) extends MTree[D, U, L]

case class Act[D, U, L](d: D, t: MTree[D, U, L])(implicit act: (D, U) => U) extends MTree[D, U, L] {
  def u: U = act(d, t.u)
}

import cats._

import cats.implicits._


defined trait Action
defined trait MTree
defined class Leaf
defined class Concat
defined class Act

In [14]:
trait Prim

case class Circle(r: Double) extends Prim
case class Rectangle(w: Double, h: Double) extends Prim

type Diagram = List[Prim]

val demo1 = List(Circle(1), Rectangle(2, 3))
val demo2 = List(Circle(4))
demo1 |+| demo2

defined trait Prim
defined class Circle
defined class Rectangle
defined type Diagram
demo1: List[Product with Serializable with Prim] = List(
  Circle(1.0),
  Rectangle(2.0, 3.0)
)
demo2: List[Circle] = List(Circle(4.0))
res13_6: List[Product with Serializable with Prim] = List(
  Circle(1.0),
  Rectangle(2.0, 3.0),
  Circle(4.0)
)

In [24]:
case class V2(x: Double, y: Double) {
  lazy val len: Double = math.sqrt(x * x + y * y)
  
  def *(m: Double) = V2(x * m, y * m)
}

case class P2(x: Double, y: Double) {
  def translate(v: V2): P2 = P2(x + v.x, y + v.y)
}

type Envelope = V2 => Double

trait Segment {
  def translate(v: V2): Segment
  
  def envelope: Envelope
}

case class LineSegment(p0: P2, p1: P2) extends Segment {
  def translate(v: V2): Segment = LineSegment(p0.translate(v), p1.translate(v))
  
  def envelope: Envelope = { case V2(vx, vy) =>
    val d0 = p0.x * vx + p0.y * vy
    val d1 = p1.x * vx + p1.y * vy
    (d0 max d1) / (vx * vx + vy * vy)
  }
}

// TODO ArcSegment, QBezSegment, CBezSegment

trait Diagram {
  def translate(v: V2): Diagram
  
  def envelope: Option[Envelope]
  
  def over(d: Diagram): Diagram = Over(this, d)
  
  def beside(v: V2, d: Diagram): Diagram = (envelope, d.envelope) match {
    case (Some(te), Some(de)) => {
      val vtrans = v * (te(v) + de(v * -1))
      over(d.translate(vtrans))
    }
    case _ => over(d)
  }
}

case object Empty extends Diagram {
  def translate(v: V2): Diagram = this
  
  val envelope: Option[Envelope] = None
  
  override def over(d: Diagram): Diagram = d
}

case class Path(segments: List[Segment]) extends Diagram {
  def translate(v: V2): Diagram = Path(segments.map(_.translate(v)))
  
  lazy val envelope: Option[Envelope] = Some(v => segments.map(_.envelope(v)).max)
}

case class Over(a: Diagram, b: Diagram) extends Diagram {
  def translate(v: V2): Diagram = Over(a.translate(v), b.translate(v))
  
  lazy val envelope: Option[Envelope] = (a.envelope, b.envelope) match {
    case (Some(ae), Some(be)) => Some(v => ae(v) max be(v))
    case (e, None) => e
    case (None, e) => e
    case _ => None
  }
}

object Diagram {
  implicit val diagramMonoid: Monoid[Diagram] =
    new Monoid[Diagram] {
      def empty: Diagram = Empty
      
      def combine(a: Diagram, b: Diagram): Diagram = a.over(b)
    }
}

defined class V2
defined class P2
defined type Envelope
defined trait Segment
defined class LineSegment
defined trait Diagram
defined object Empty
defined class Path
defined class Over
defined object Diagram

In [25]:
def rectangle(w: Double, h: Double): Diagram = {
  val ul = P2(-w/2, h/2)
  val ur = P2(w/2, h/2)
  val ll = P2(-w/2, -h/2)
  val lr = P2(w/2, -h/2)
  Path(List(
    LineSegment(ul, ur),
    LineSegment(ur, lr),
    LineSegment(lr, ll),
    LineSegment(ll, ul)
  ))
}

val r1 = rectangle(2, 3)
val Some(e) = r1.envelope
e(V2(2, 0))
e(V2(2, 1))
e(V2(2, 2))
e(V2(2, 3))
e(V2(1, 3))
e(V2(0, 3))
e(V2(-1, 3))
e(V2(2, -1))
e(V2(2, -2))
e(V2(2, -3))
e(V2(1, -3))
e(V2(0, -3))

defined function rectangle
r1: Diagram = Path(
  List(
    LineSegment(P2(-1.0, 1.5), P2(1.0, 1.5)),
    LineSegment(P2(1.0, 1.5), P2(1.0, -1.5)),
    LineSegment(P2(1.0, -1.5), P2(-1.0, -1.5)),
    LineSegment(P2(-1.0, -1.5), P2(-1.0, 1.5))
  )
)
e: V2 => Double = ammonite.$sess.cmd23$Helper$Path$$Lambda$3507/0x00000008019d3840@1e4c70f7
res24_3: Double = 0.5
res24_4: Double = 0.7
res24_5: Double = 0.625
res24_6: Double = 0.5
res24_7: Double = 0.55
res24_8: Double = 0.5
res24_9: Double = 0.55
res24_10: Double = 0.7
res24_11: Double = 0.625
res24_12: Double = 0.5
res24_13: Double = 0.55
res24_14: Double = 0.5

In [26]:
val r2 = rectangle(1, 4)
val d2 = r1 |+| r2
val Some(e2) = d2.envelope
e2(V2(2, 0))
e2(V2(2, 1))
e2(V2(2, 2))
e2(V2(2, 3))
e2(V2(1, 3))
e2(V2(0, 3))
e2(V2(-1, 3))
e2(V2(2, -1))
e2(V2(2, -2))
e2(V2(2, -3))
e2(V2(1, -3))
e2(V2(0, -3))

r2: Diagram = Path(
  List(
    LineSegment(P2(-0.5, 2.0), P2(0.5, 2.0)),
    LineSegment(P2(0.5, 2.0), P2(0.5, -2.0)),
    LineSegment(P2(0.5, -2.0), P2(-0.5, -2.0)),
    LineSegment(P2(-0.5, -2.0), P2(-0.5, 2.0))
  )
)
d2: Diagram = Over(
  Path(
    List(
      LineSegment(P2(-1.0, 1.5), P2(1.0, 1.5)),
      LineSegment(P2(1.0, 1.5), P2(1.0, -1.5)),
      LineSegment(P2(1.0, -1.5), P2(-1.0, -1.5)),
      LineSegment(P2(-1.0, -1.5), P2(-1.0, 1.5))
    )
  ),
  Path(
    List(
      LineSegment(P2(-0.5, 2.0), P2(0.5, 2.0)),
      LineSegment(P2(0.5, 2.0), P2(0.5, -2.0)),
      LineSegment(P2(0.5, -2.0), P2(-0.5, -2.0)),
      LineSegment(P2(-0.5, -2.0), P2(-0.5, 2.0))
    )
  )
)
e2: V2 => Double = ammonite.$sess.cmd23$Helper$Over$$Lambda$3539/0x00000008019e7040@2febdc3c
res25_3: Double = 0.5
res25_4: Double = 0.7
res25_5: Double = 0.625
res25_6: Double = 0.5384615384615384
res25_7: Double = 0.65
res25_8: Double = 0.6666666666666666
res25_9: Double = 0.65
res25_10: Double = 0.7
res25_

In [27]:
val d3 = r1.beside(V2(1, 0), r2)
val Some(e3) = d3.envelope
e3(V2(2, 0))
e3(V2(2, 1))
e3(V2(2, 2))
e3(V2(2, 3))
e3(V2(1, 3))
e3(V2(0, 3))
e3(V2(-1, 3))
e3(V2(2, -1))
e3(V2(2, -2))
e3(V2(2, -3))
e3(V2(1, -3))
e3(V2(0, -3))

d3: Diagram = Over(
  Path(
    List(
      LineSegment(P2(-1.0, 1.5), P2(1.0, 1.5)),
      LineSegment(P2(1.0, 1.5), P2(1.0, -1.5)),
      LineSegment(P2(1.0, -1.5), P2(-1.0, -1.5)),
      LineSegment(P2(-1.0, -1.5), P2(-1.0, 1.5))
    )
  ),
  Path(
    List(
      LineSegment(P2(1.0, 2.0), P2(2.0, 2.0)),
      LineSegment(P2(2.0, 2.0), P2(2.0, -2.0)),
      LineSegment(P2(2.0, -2.0), P2(1.0, -2.0)),
      LineSegment(P2(1.0, -2.0), P2(1.0, 2.0))
    )
  )
)
e3: V2 => Double = ammonite.$sess.cmd23$Helper$Over$$Lambda$3539/0x00000008019e7040@31547d86
res26_2: Double = 1.0
res26_3: Double = 1.2
res26_4: Double = 1.0
res26_5: Double = 0.7692307692307693
res26_6: Double = 0.8
res26_7: Double = 0.6666666666666666
res26_8: Double = 0.55
res26_9: Double = 1.2
res26_10: Double = 1.0
res26_11: Double = 0.7692307692307693
res26_12: Double = 0.8
res26_13: Double = 0.6666666666666666